# Assignment 01 - HW on Gym
CMPE 260 Homework 1
By Shawn Chumbar

## Assignment Description
Use the Q-learning example given as a basis for your homework and develop a
SARSA learning algorithm instead of Q-learning.


### Setup

In [ ]:
!pip install gymnasium[atari]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.5 MB/s eta 0:00:00


In [ ]:
!pip install "ale-py>=0.8.0"

In [ ]:
!pip install autorom[accept-rom-license]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 27.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446661 sha256=4587a0c38834593f96b438cadc3ede1ef46edb6e76bc697f43f990d48c9ccb3d
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


In [ ]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo

In [ ]:
def run_gym_env(name, n):
    env = gym.make(name, render_mode="rgb_array")
    env = RecordVideo(env, "videos")
    observation, info = env.reset()
    for _ in range(int(n)):
        action = env.action_space.sample()
        observation, reward, terminated, truncated, info = env.step(action)
        if terminated or truncated:
            observation, info = env.reset()
    env.close()

if __name__ == "__main__":
    run_gym_env('ALE/Alien-v5', 2000)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Moviepy - Building video /content/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-0.mp4
Moviepy - Building video /content/videos/rl-video-episode-1.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-1.mp4


### Q_Learner
The code below is the standard Q Learner class that we were given in Class.

In [ ]:
import gymnasium as gym
import numpy as np

MAX_NUM_EPISODES = 50000
STEPS_PER_EPISODE = 200
EPSILON_MIN = 0.005
max_num_steps = MAX_NUM_EPISODES * STEPS_PER_EPISODE
EPSILON_DECAY = 500 * EPSILON_MIN / max_num_steps
ALPHA = 0.05
GAMMA = 0.98
NUM_DISCRETE_BINS = 30

class Q_Learner(object):
    def __init__(self, env):
        self.obs_shape = env.observation_space.shape
        self.obs_high = env.observation_space.high
        self.obs_low = env.observation_space.low
        self.obs_bins = NUM_DISCRETE_BINS
        self.bin_width = (self.obs_high - self.obs_low) / self.obs_bins
        self.action_shape = env.action_space.n
        self.Q = np.zeros((self.obs_bins + 1, self.obs_bins + 1, self.action_shape))
        self.alpha = ALPHA
        self.gamma = GAMMA
        self.epsilon = 1.0

    def discretize(self, obs):
        return tuple(((obs - self.obs_low) / self.bin_width).astype(int))

    def get_action(self, obs):
        discretized_obs = self.discretize(obs)
        if self.epsilon > EPSILON_MIN:
            self.epsilon -= EPSILON_DECAY
        if np.random.random() > self.epsilon:
            return np.argmax(self.Q[discretized_obs])
        else:
            return np.random.choice([a for a in range(self.action_shape)])

    def learn(self, obs, action, reward, next_obs):
        discretized_obs = self.discretize(obs)
        discretized_next_obs = self.discretize(next_obs)
        td_target = reward + self.gamma * np.max(self.Q[discretized_next_obs])
        td_error = td_target - self.Q[discretized_obs][action]
        self.Q[discretized_obs][action] += self.alpha * td_error

def train(agent, env):
    best_reward = -float('inf')
    for episode in range(MAX_NUM_EPISODES):
        obs, _ = env.reset()
        total_reward = 0.0
        for _ in range(STEPS_PER_EPISODE):
            action = agent.get_action(obs)
            next_obs, reward, terminated, truncated, _ = env.step(action)
            agent.learn(obs, action, reward, next_obs)
            obs = next_obs
            total_reward += reward
            if terminated or truncated:
                break
        if total_reward > best_reward:
            best_reward = total_reward
        if episode % 100 == 0:
            print(f"Episode#{episode} reward:{total_reward} best_reward:{best_reward} eps:{agent.epsilon:.3f}")
    return np.argmax(agent.Q, axis=2)

def test(agent, env, policy):
    obs, _ = env.reset()
    total_reward = 0.0
    for _ in range(STEPS_PER_EPISODE):
        action = policy[agent.discretize(obs)]
        next_obs, reward, terminated, truncated, _ = env.step(action)
        obs = next_obs
        total_reward += reward
        if terminated or truncated:
            break
    return total_reward

if __name__ == "__main__":
    env = gym.make('MountainCar-v0')
    agent = Q_Learner(env)
    learned_policy = train(agent, env)

    test_env = gym.make('MountainCar-v0', render_mode='rgb_array')
    test_env = gym.wrappers.RecordVideo(test_env, "videos")
    for _ in range(5):  # Test for 5 episodes
        total_reward = test(agent, test_env, learned_policy)
        print(f"Test episode reward: {total_reward}")
    test_env.close()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Episode#0 reward:-200.0 best_reward:-200.0 eps:1.000
Episode#100 reward:-200.0 best_reward:-200.0 eps:0.995
Episode#200 reward:-200.0 best_reward:-200.0 eps:0.990
Episode#300 reward:-200.0 best_reward:-200.0 eps:0.985
Episode#400 reward:-200.0 best_reward:-200.0 eps:0.980
Episode#500 reward:-200.0 best_reward:-200.0 eps:0.975
Episode#600 reward:-200.0 best_reward:-200.0 eps:0.970
Episode#700 reward:-200.0 best_reward:-200.0 eps:0.965
Episode#800 reward:-200.0 best_reward:-200.0 eps:0.960
Episode#900 reward:-200.0 best_reward:-200.0 eps:0.955
Episode#1000 reward:-200.0 best_reward:-200.0 eps:0.950
Episode#1100 reward:-200.0 best_reward:-200.0 eps:0.945
Episode#1200 reward:-200.0 best_reward:-200.0 eps:0.940
Episode#1300 reward:-200.0 best_reward:-200.0 eps:0.935
Episode#1400 reward:-200.0 best_reward:-200.0 eps:0.930
Episode#1500 reward:-200.0 best_reward:-200.0 eps:0.925
Episode#1600 reward:-200.0 best_reward:-200.0 eps:0.920
Episode#1700 reward:-200.0 best_reward:-200.0 eps:0.915
Epis

/usr/local/lib/python3.10/dist-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /content/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video /content/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-0.mp4
Test episode reward: -141.0
Moviepy - Building video /content/videos/rl-video-episode-1.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-1.mp4
Test episode reward: -140.0
Test episode reward: -142.0
Test episode reward: -150.0
Test episode reward: -124.0


### SARSA Learner

In [ ]:
import gymnasium as gym
import numpy as np

MAX_NUM_EPISODES = 50000
STEPS_PER_EPISODE = 200
EPSILON_MIN = 0.005
max_num_steps = MAX_NUM_EPISODES * STEPS_PER_EPISODE
EPSILON_DECAY = 500 * EPSILON_MIN / max_num_steps
ALPHA = 0.05
GAMMA = 0.98
NUM_DISCRETE_BINS = 30

class SARSA_Learner(object):
    def __init__(self, env):
        self.obs_shape = env.observation_space.shape
        self.obs_high = env.observation_space.high
        self.obs_low = env.observation_space.low
        self.obs_bins = NUM_DISCRETE_BINS
        self.bin_width = (self.obs_high - self.obs_low) / self.obs_bins
        self.action_shape = env.action_space.n
        self.Q = np.zeros((self.obs_bins + 1, self.obs_bins + 1, self.action_shape))
        self.alpha = ALPHA
        self.gamma = GAMMA
        self.epsilon = 1.0

    def discretize(self, obs):
        return tuple(((obs - self.obs_low) / self.bin_width).astype(int))

    def get_action(self, obs):
        discretized_obs = self.discretize(obs)
        if self.epsilon > EPSILON_MIN:
            self.epsilon -= EPSILON_DECAY
        if np.random.random() > self.epsilon:
            return np.argmax(self.Q[discretized_obs])
        else:
            return np.random.choice([a for a in range(self.action_shape)])

    # Modified learn method for SARSA
    def learn(self, obs, action, reward, next_obs, next_action):
        discretized_obs = self.discretize(obs)
        discretized_next_obs = self.discretize(next_obs)
        # SARSA Update to use Q-value of next state-action pair instead of max Q-value
        td_target = reward + self.gamma * self.Q[discretized_next_obs][next_action]
        td_error = td_target - self.Q[discretized_obs][action]
        self.Q[discretized_obs][action] += self.alpha * td_error

def train(agent, env):
    best_reward = -float('inf')
    for episode in range(MAX_NUM_EPISODES):
        obs, _ = env.reset()
        total_reward = 0.0
        # Choose initial action
        action = agent.get_action(obs)  # Added this line for SARSA
        for _ in range(STEPS_PER_EPISODE):
            next_obs, reward, terminated, truncated, _ = env.step(action)
            next_action = agent.get_action(next_obs)  # Choose next action
            # SARSA learning: pass current state-action and next state-action
            agent.learn(obs, action, reward, next_obs, next_action)
            obs = next_obs
            action = next_action  # Update action for next iteration
            total_reward += reward
            if terminated or truncated:
                break
        if total_reward > best_reward:
            best_reward = total_reward
        if episode % 100 == 0:
            print(f"Episode#{episode} reward:{total_reward} best_reward:{best_reward} eps:{agent.epsilon:.3f}")
    return np.argmax(agent.Q, axis=2)

def test(agent, env, policy):
    obs, _ = env.reset()
    total_reward = 0.0
    for _ in range(STEPS_PER_EPISODE):
        action = policy[agent.discretize(obs)]
        next_obs, reward, terminated, truncated, _ = env.step(action)
        obs = next_obs
        total_reward += reward
        if terminated or truncated:
            break
    return total_reward

if __name__ == "__main__":
    env = gym.make('MountainCar-v0')
    agent = SARSA_Learner(env)  # Changed to SARSA_Learner
    learned_policy = train(agent, env)

    test_env = gym.make('MountainCar-v0', render_mode='rgb_array')
    test_env = gym.wrappers.RecordVideo(test_env, "videos")
    for _ in range(20):  # Test for 5 episodes
        total_reward = test(agent, test_env, learned_policy)
        print(f"Test episode reward: {total_reward}")
    test_env.close()

Episode#0 reward:-200.0 best_reward:-200.0 eps:1.000
Episode#100 reward:-200.0 best_reward:-200.0 eps:0.995
Episode#200 reward:-200.0 best_reward:-200.0 eps:0.990
Episode#300 reward:-200.0 best_reward:-200.0 eps:0.985
Episode#400 reward:-200.0 best_reward:-200.0 eps:0.980
Episode#500 reward:-200.0 best_reward:-200.0 eps:0.975
Episode#600 reward:-200.0 best_reward:-200.0 eps:0.970
Episode#700 reward:-200.0 best_reward:-200.0 eps:0.965
Episode#800 reward:-200.0 best_reward:-200.0 eps:0.960
Episode#900 reward:-200.0 best_reward:-200.0 eps:0.955
Episode#1000 reward:-200.0 best_reward:-200.0 eps:0.950
Episode#1100 reward:-200.0 best_reward:-200.0 eps:0.945
Episode#1200 reward:-200.0 best_reward:-200.0 eps:0.940
Episode#1300 reward:-200.0 best_reward:-200.0 eps:0.935
Episode#1400 reward:-200.0 best_reward:-200.0 eps:0.930
Episode#1500 reward:-200.0 best_reward:-200.0 eps:0.925
Episode#1600 reward:-200.0 best_reward:-200.0 eps:0.920
Episode#1700 reward:-200.0 best_reward:-200.0 eps:0.915
Epis

Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-0.mp4
Test episode reward: -111.0
Moviepy - Building video /content/videos/rl-video-episode-1.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-1.mp4
Test episode reward: -110.0
Test episode reward: -100.0
Test episode reward: -148.0
Test episode reward: -147.0
Test episode reward: -111.0
Test episode reward: -109.0
Test episode reward: -111.0
Moviepy - Building video /content/videos/rl-video-episode-8.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-8.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-8.mp4
Test episode reward: -111.0
Test episode reward: -111.0
Test episode reward: -97.0
Test episode reward: -100.0
Test episode reward: -111.0
Test episode reward: -112.0
Test episode reward: -111.0
Test episode reward: -156.0
Test episode reward: -143.0
Test episode reward: -100.0
Test episode reward: -153.0
Test episode reward: -112.0


# Results

In this project, two reinforcement learning algorithms, Q-Learning and SARSA, were applied to solve the Mountain Car problem. The results demonstrate distinct learning patterns and final performances for each method, illustrating their unique strengths and weaknesses.



#### Q-Learning vs. SARSA Performance

1. **Learning Behavior:**
   - **Q-Learning:** In the early stages, Q-Learning exhibited faster learning due to its off-policy nature, which allows the agent to maximize long-term rewards by considering actions outside the current policy. This led to more aggressive exploration, yielding higher episodic rewards earlier in the training.
   - **SARSA:** On the other hand, SARSA, being an on-policy method, tended to be more conservative in its exploration. This approach made the learning process more stable but resulted in slower initial progress compared to Q-Learning. SARSA gradually improved its performance as the number of episodes increased.

2. **Convergence and Stability:**
   - **Q-Learning** displayed more fluctuation in rewards as it focused on maximizing future rewards even at the risk of taking suboptimal actions during training. Despite these fluctuations, Q-Learning converged faster in terms of finding a solution that allowed the car to reach the top of the hill.
   - **SARSA** had a smoother learning curve with fewer fluctuations, showing a more stable convergence pattern. This is attributed to SARSA's conservative nature, which ensures it sticks closely to the policy being evaluated, reducing the chances of drastic changes in learned behavior.

3. **Exploration and Exploitation:**
   - The exploration rate (epsilon) decreased steadily in both methods, indicating that both algorithms reduced exploration over time as the policy became more stable. By the end of the training, both Q-Learning and SARSA were operating with minimal exploration, focusing on exploiting the learned policy.



#### Conclusion

The results show that Q-Learning outperforms SARSA in terms of speed of convergence due to its off-policy nature, which allows for more aggressive learning. However, SARSA's on-policy approach results in a more stable learning process, which may be preferable in environments where stability is critical. Overall, the choice between Q-Learning and SARSA should be guided by the specific requirements of the problem at hand, such as the need for speed or stability in learning.

This analysis highlights the trade-offs between these two algorithms, providing insights into their behavior and suitability for solving reinforcement learning problems.